In [1]:
import requests
import pandas as pd 
import json
import concurrent.futures
import re
import queue
import os
from bs4 import BeautifulSoup
import lxml

from collections import defaultdict
import unicodedata

In [2]:
__name = pd.read_csv("allocate/VĂN HỌC.csv")
__name.columns

Index(['tên sách', 'ảnh bìa', 'thể loại', 'tác giả', 'nội dung tóm tắt',
       'giá bìa', 'công ty phát hành', 'nhà xuất bản', 'ngày xuất bản',
       'kích thước', 'loại bìa', 'số trang', 'dịch giả', 'path', 'phiên bản',
       'thương hiệu', 'xuất xứ', 'xuất xứ thương hiệu'],
      dtype='object')

In [3]:
df2 = pd.read_csv("DATA_v2.csv")
df2.dropna(inplace = True) 
df2.columns = ["Thể loại", "Nguồn nhập", "type"]

# Remove all row that not have fahasa
df2 = df2[df2['Nguồn nhập'].str.contains("vina")]
group_f1 = df2.groupby("Thể loại")["Nguồn nhập"].apply(list).to_dict()
group_f2 = df2.groupby("type")["Thể loại"].apply(list).to_dict()

n_dict = defaultdict(dict)
for k, v in group_f2.items():
    for stype in v:
        if type(group_f1[stype]) == list:
            _url = group_f1[stype][0]
        else:
            _url = group_f1[stype]
            
        n_dict[k][stype] = group_f1[stype][0] + 'page-{}/?sef_rewrite=1'

In [43]:
n_dict

defaultdict(dict,
            {'CHĂM SÓC GIA ĐÌNH': {'Hạnh phúc gia đình': 'https://www.vinabook.com/c513/hanh-phuc-gia-dinh/page-{}/?sef_rewrite=1'},
             'HỌC NGÔN NGỮ': {'Tiếng Trung': 'https://www.vinabook.com/c412/tieng-hoa/page-{}/?sef_rewrite=1',
              'Tiếng Pháp ': 'https://www.vinabook.com/c411/tieng-phap/page-{}/?sef_rewrite=1',
              'Tiếng Nhật ': 'https://www.vinabook.com/c413/tieng-nhat-1/page-{}/?sef_rewrite=1'},
             'NGÀNH CÔNG NGHỆ - THÔNG TIN': {'Cơ sở dữ liệu': 'https://www.vinabook.com/c399/co-so-du-lieu/page-{}/?sef_rewrite=1',
              'Hệ điều hành': 'https://www.vinabook.com/c400/he-dieu-hanh/page-{}/?sef_rewrite=1',
              'Internet - mạng': 'https://www.vinabook.com/c398/internet-mang/page-{}/?sef_rewrite=1',
              'Lập trình - Phần mềm ứng dụng': 'https://www.vinabook.com/c397/lap-trinh-phan-mem-ung-dung/page-{}/?sef_rewrite=1',
              'Lập trình web': ' https://www.vinabook.com/c396/lap-trinh-web/p

In [4]:
url = "https://www.vinabook.com/c495/co-vua-co-tuong/page-2/?sef_rewrite=1"

payload={}
headers = {
  'authority': 'www.vinabook.com',
  'pragma': 'no-cache',
  'cache-control': 'no-cache',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'sec-fetch-site': 'none',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-user': '?1',
  'sec-fetch-dest': 'document',
  'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'accept-language': 'vi,en-US;q=0.9,en;q=0.8,vi-VN;q=0.7',
}

response = requests.request("GET", url, headers=headers, data=payload)

In [5]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
_books = soup.findAll("p",{"class":"price-info-nd"})
_books[0].a['href'].rstrip()

' https://www.vinabook.com/ma-phao-van-bien-p41969.html'

In [7]:
int(soup.find("span",{"class":"group-paging-label"}).text.split("/")[1])

5

In [8]:
_detail = requests.request("GET", _books[0].a['href'].rstrip(), headers=headers, data=payload)
_soup = BeautifulSoup(_detail.text, "html.parser")

In [17]:
_soup.h1.text

'Mã Pháo Vân Biên'

In [21]:
_soup.find("div",{"class":"bk-cover"}).img['src']

'https://www.vinabook.com/images/thumbnails/product/240x/35426_41969.jpg'

In [34]:
_soup.find("div",{"class":"cm-image-wrap"}).img['src']

'https://www.vinabook.com/images/thumbnails/product/240x/35426_41969.jpg'

In [23]:
_soup.find("div",{"class":"full-description"}).text.split("...")[0]

'  Thông tin tác giả    Lý Anh Mậu     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả       Đào Bá Kiều     Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Cờ Thế và Cờ Tàn trong nghệ thuật Tượng Kỳ, có người cho rằng đó là tinh hoa tuyệt hảo của thánh hiền truyền lại cho hậu thế, hoặc do những trí hóa siêu kì của nhiều lớp kỳ thủ đúc kết qua nhiều thế kỉ.Sách dẫn giải các thế cờ, biên soạn thêm các bài bản và các khúc biến để bổ sung cho tài liệu của các trận đồ Cờ Thế.Long môn trậnBát nam tấn bửuĐăng đàn bái tướngVạn lý trường thànhMãi quốc cầu vinhThiên lý mãVạn tiên trậnChư tiên trậnPhi hồ quá quanDực mão tam quan'

In [24]:
_soup.find("span",{"class":"list-price nowrap"}).text

'20.000'

In [31]:
_specific = _soup.find("div",{"class":"product-feature"})
_information = {}
for _ in _specific.findAll("li"):
#     print(_)
    _regex = re.sub('\s+',' ',_.text).split(":")
    _information[_regex[0]] = _regex[1]
    print(_.text.strip())

Tác giả:  Lý Anh Mậu
                            ,                                                     Đào Bá Kiều
Nhà xuất bản:  NXB Đồng Nai
Nhà phát hành:  Thị Nghè
Khối lượng: 
            150.00 gam
Định dạng: 
                
                    
                        
                    
                        
                    
                
                                        Bìa mềm
Kích thước: 
                
                    
                        
                    
                        
                    
                
                                        14.5 x 20.5 cm
Ngày phát hành:  
                        10/2010
Số trang:  132


In [32]:
_information

{' Tác giả': ' Lý Anh Mậu , Đào Bá Kiều ',
 ' Nhà xuất bản': ' NXB Đồng Nai ',
 ' Nhà phát hành': ' Thị Nghè ',
 ' Khối lượng': ' 150.00 gam ',
 ' Định dạng': ' Bìa mềm ',
 ' Kích thước': ' 14.5 x 20.5 cm ',
 ' Ngày phát hành': ' 10/2010 ',
 ' Số trang': ' 132 '}

In [30]:
re.sub('\s+',' ',_specific.findAll("li")[0].text)

' Tác giả: Lý Anh Mậu , Đào Bá Kiều '

In [35]:
_big_dict = {}
booksQueue = queue.Queue()

class Crawler():
    def __init__(self, url, cname):
        self.url = url
        self.category_name = cname
        
    def crawl(self):
        _concurrent = self._bookshelf()
        
        print(_concurrent)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()*3) as executor:
            executor.map(self._books, _concurrent)
            
    def _books(self, _url,):
        
        book = {}
        __res =  requests.request("GET",_url, headers=headers)
        _soup = BeautifulSoup(__res.text, "html.parser")
        
        book['tên sách'] = _soup.h1.text
        book['ảnh bìa'] = _soup.find("div",{"class":"cm-image-wrap"}).img['src']
        book['thể loại'] = self.category_name
        
        _specific = _soup.find("div",{"class":"product-feature"})
        for _ in _specific.findAll("li"):
            _regex = re.sub('\s+',' ',_.text).split(":")
            book[_regex[0].lower()] = _regex[1]
            
        book['nội dung tóm tắt'] = _soup.find("div",{"class":"full-description"}).text.split("...")[0]
        book['giá bìa'] = _soup.find("span",{"class":"list-price nowrap"}).text
        
        print(book)
        booksQueue.put(book)
        print(f"Number of products in my pocket {booksQueue.qsize()}")
        
    def _bookshelf(self):
        
        _bookshelf = []
        print(self.url.format(48, 1))
        _res = requests.request("GET", self.url.format(1), headers=headers)
        soup = BeautifulSoup(_res.text, "html.parser")
        _pages = int(soup.find("span",{"class":"group-paging-label"}).text.split("/")[1]) + 1
        
        for _ in range(_pages):
            _ =+1
#             print(f"total page of {self.urlkey} : {_total}")
            _res = requests.request("GET", self.url.format(_), headers=headers)
            soup = BeautifulSoup(_res.text, "html.parser")
            for _ in soup.findAll("p",{"class":"price-info-nd"}):
                _bookshelf.append(_.a['href'].rstrip())
            
        return _bookshelf

In [36]:
t = Crawler('https://www.vinabook.com/c430/truyen-lich-su/page-{}/?sef_rewrite=1','test')
# print(t._bookshelf())
print(t._books('https://www.vinabook.com/tam-quoc-dien-nghia-bo-3-cuon--p93877.html'))

{'tên sách': 'Tam Quốc Diễn Nghĩa ( Bộ 3 Cuốn )', 'ảnh bìa': 'https://www.vinabook.com/images/thumbnails/product/240x/366475_p93877mboxsetthreekingdom15a694c8b528649598fdf9b2d262821c2small.jpg', 'thể loại': 'test', ' tác giả': ' La Quán Trung ', ' nhà xuất bản': ' Nxb văn học ', ' nhà phát hành': ' Bestbooks ', ' mã sản phẩm': ' 9786049937934 ', ' khối lượng': ' 2300.00 gam ', ' ngôn ngữ': ' Tiếng Việt ', ' định dạng': ' Bìa mềm ', ' kích thước': ' 14.5 x 20.5 cm ', ' ngày phát hành': ' 2020 ', ' số trang': ' 1900 ', 'nội dung tóm tắt': '  Thông tin tác giả    La Quán Trung \n                                                                                                                                            La Quán Trung\n                                                                   Vào trang riêng của tác giả   Xem tất cả các sách của tác giả     Tam Quốc Diễn Nghĩa (Trọn Bộ 3 Tập) Được vinh danh là một trong “Tứ đại danh tác” hay “Tứ đại kỳ thư” của Trung Quốc, cho đến nay

In [38]:
for _name in n_dict.keys():
    booksQueue = queue.Queue()
    for k,v in n_dict[_name].items():
        print(v)
        t = Crawler(v, k)
        t.crawl()

    total += booksQueue.qsize()
    df = pd.DataFrame(columns=__name.columns,index=[0])
    while not booksQueue.empty():
        df = df.append(booksQueue.get(),ignore_index=True)

    df.to_csv('vina/fahasa_'+ _name+'.csv')

https://www.vinabook.com/c513/hanh-phuc-gia-dinh/page-{}/?sef_rewrite=1
https://www.vinabook.com/c412/tieng-hoa/page-{}/?sef_rewrite=1
https://www.vinabook.com/c411/tieng-phap/page-{}/?sef_rewrite=1
https://www.vinabook.com/c413/tieng-nhat-1/page-{}/?sef_rewrite=1
https://www.vinabook.com/c399/co-so-du-lieu/page-{}/?sef_rewrite=1
https://www.vinabook.com/c400/he-dieu-hanh/page-{}/?sef_rewrite=1
https://www.vinabook.com/c398/internet-mang/page-{}/?sef_rewrite=1
https://www.vinabook.com/c397/lap-trinh-phan-mem-ung-dung/page-{}/?sef_rewrite=1
 https://www.vinabook.com/c396/lap-trinh-web/page-{}/?sef_rewrite=1
https://www.vinabook.com/c401/phan-cung/page-{}/?sef_rewrite=1
https://www.vinabook.com/c394/tin-hoc-co-ban/page-{}/?sef_rewrite=1
https://www.vinabook.com/c395/tin-hoc-van-phong/page-{}/?sef_rewrite=1
https://www.vinabook.com/c402/do-hoa/page-{}/?sef_rewrite=1
https://www.vinabook.com/c403/sach-tin-hoc-bang-ngoai-ngu/page-{}/?sef_rewrite=1
https://www.vinabook.com/c447/cau-duong/pag